<a id='logbook'></a>
# Summary 2020-12-15

In [ ]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables


## Nomenclature
| Variable | Explain |
|---|---|
|$\pi$| example |

Here is a cell link: [Logbook](#logbook)

# Abstract

Many cost-efficient computation methods have been developed over the years to analyze various aspects of ship hydrodynamics such as:  resistance,  propulsion  and  seakeeping. Getting the  best  possible  accuracy with the lowest possible computational cost is an important factor in a ship’s early design stage.  Potential flow-based analysis partly presents such a solution for seakeeping, with good accuracy for heave and pitch, but not for roll where the roll damping contains both inviscid and viscouseffects.  Roll motion is, however, often a critical degree of freedom that needs to be analyzed since large roll motions can result in cargo shifting or even capsizing. The viscous part of roll damping can be assessed with high accuracy by means of experimental model tests or URANS calculations, but these are generally too expensive in the early design stage of ships. Many semi-empirical formulas to determine viscous damping were therefore developed during the 1970s, where Ikeda’s method is one of the most widely used.  The viscous damping from this method is normally combined with inviscid roll damping from strip theory.

With today’s computational power, more advanced potential flow methods can be used in the seakeeping analysis to enhance the accuracy in the predictions, but still at relatively low computational cost.  This paper investigates the feasibility of combining 3D unsteady fully nonlinearpotential flow (FNPF) theory solved by means of a Boundary ElementMethod  (BEM)  together  with  the  viscous  contributions  from  Ikeda’smethod.

The approach of substituting the inviscid part from Ikeda’s method using strip theory with FNPF is investigated by conducting roll decay simulations. The results estimated by the proposed approach are compared with both the classic strip theory approach and roll decay model tests. 
**It is found that potential improvements to the modelling of roll damping can be achieved by introducing FNPF analysis in the Ikeda’s method.**

# Abstract

* **Important**: Good accuracy at low computational cost. 

(URANS or model test too expensive)

* &rarr; seakeeping: Potential flow

* heave/pitch good!

* **NOT** roll!

* **Solution**: Potential flow + semi empirical viscous roll damping

* Potential flow:
    * Strip theory (milli seconds)
    * Nonlinear 3D methods (hours)
    

## Background
The roll damping can be divided into various components <cite data-cite="7505983/4AFVVGNT"></cite>:
$$B_{44} = B_F + B_E + B_L + B_W + B_{BK}$$

Viscous: $ B_{visc} = B_F + B_E + B_L + B_{BK} $ (Ikeda's method, Simplified Ikeda)

Inviscid: $ B_{invisc} = B_W $ (Potential flow)

$$B^{Ikeda} = B_{invisc}^{1D} + B_{visc}$$

$$B^{Motions} = B_{invisc}^{3D} + B_{visc}$$

### Problems with $B_W$ in Simplified Ikeda
![](SI_BW.png)

In [ ]:
data = {
    'KVLCC2' : {
        'type':'tanker',
        'test data':True,
        'B_W': 'small',
        'bilge keel':False,
        'publish geom':True,
        'publish test':True,
    },
    'DTC' : {
        'type':'container',
        'test data':True,
        'B_W': '?',
        'bilge keel':True,
        'publish geom':True,
        'publish test':True,
    },
    'Wallenius' : {
        'type':'PCTC',
        'test data':True,
        'B_W': 'medium',
        'bilge keel':True,
        'publish geom':False,
        'publish test':True,
    },
    
}
test_cases = pd.DataFrame(data=data).transpose()

In [ ]:
def background_colorer(val): 
    return 'background-color: %s' % get_color(val)

def text_colorer(val): 
    return 'color: %s' % get_color(val)

def get_color(val):
    
    color = 'none'
    if isinstance(val, bool):
        if val:
            color = 'green'
        else:
            color = 'red'
            
    return color


## Possible test cases:

In [ ]:
test_cases.style.applymap(background_colorer).applymap(text_colorer)

## Test case: KVLCC2 
[04.3_KVLCC2_Ikedas_model_tests](../../notebooks/04.3_KVLCC2_Ikedas_model_tests.ipynb)

<img src="../../reports/figures/KVLCC2_B_e_0.0.png" width="1100">

* B_W very small!
* B_E is dominating

<img src="../../reports/figures/KVLCC2_B_e_15.5.png" width="1100">

* B_E decreased, B_L is now dominating
* B_W is a bit larger but still minor

# Abstract conclusion:
#### "It is found that potential improvements to the modelling of roll damping can be achieved by introducing FNPF analysis in the Ikeda’s method"

# Is this what we are aiming for?

## References
<div class="cite2c-biblio"></div>